In [20]:
# Dependencies
import warnings
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool, ScrapeWebsiteTool
from pydantic import BaseModel
import openai
import yfinance as yf

# Load environment variables from .env file
load_dotenv()

True

In [21]:
# Get API keys
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME", "text-davinci-003")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

# Set API keys
openai.api_key = OPENAI_API_KEY

In [22]:
# Chatbot Agent
class ChatbotAgent(Agent):
    def __init__(self):
        super().__init__(
            role="Chatbot Agent",
            goal="Collect the company name from the user and validate its stock ticker",
            backstory="You are a friendly chatbot that helps users find information about companies.",
            verbose=True,
            allow_delegation=False,
        )

    def get_stock_ticker(self, company_name: str):
        try:
            ticker = yf.Ticker(company_name).info['symbol']
            return ticker
        except Exception as e:
            print(f"Error occurred while retrieving the stock ticker: {e}")
            return None

chatbot_agent = ChatbotAgent()

In [23]:
# News Retrieval Agent
news_tool = SerperDevTool(api_key=SERPER_API_KEY)
news_retrieval_agent = Agent(
    role="News Retrieval Agent",
    goal="Fetch the latest news about the company",
    backstory="You collect the most recent news articles related to a company for sentiment analysis.",
    verbose=True,
    tools=[news_tool],
)


In [24]:
# Sentiment Analysis Agent
class SentimentAnalysisAgent(Agent):
    def __init__(self):
        super().__init__(
            role="Sentiment Analysis Agent",
            goal="Analyze the sentiment of recent news articles for a given company",
            backstory="You are responsible for determining if the news sentiment about a company is positive, neutral, or negative.",
            verbose=True,
            allow_delegation=False,
        )

    def analyze_sentiment(self, articles):
        try:
            analysis_results = []
            for article in articles:
                response = openai.Completion.create(
                    engine=OPENAI_MODEL_NAME,
                    prompt=f"Analyze the sentiment of the following text: \n{article}",
                    max_tokens=10,
                )
                sentiment = response.choices[0].text.strip()
                analysis_results.append(sentiment)
            return analysis_results
        except Exception as e:
            print(f"Error occurred during sentiment analysis: {e}")
            return []

sentiment_analysis_agent = SentimentAnalysisAgent()


In [25]:
# Define Recommendation Agent
class RecommendationAgent(Agent):
    def __init__(self):
        super().__init__(
            role="Recommendation Agent",
            goal="Provide an investment recommendation based on the sentiment analysis results",
            backstory="You provide a recommendation to the user based on the sentiment analysis of the company's recent news articles.",
            verbose=True,
            allow_delegation=False,
        )

    def generate_recommendation(self, sentiment_scores):
        try:
            positive_count = sentiment_scores.count("positive")
            negative_count = sentiment_scores.count("negative")
            if positive_count > negative_count:
                return "Positive news sentiment—consider investment."
            elif negative_count > positive_count:
                return "Negative news sentiment—exercise caution."
            else:
                return "Mixed sentiment—recommend further investigation."
        except Exception as e:
            print(f"Error occurred during recommendation generation: {e}")
            return "Unable to generate recommendation."

recommendation_agent = RecommendationAgent()


In [26]:
# Tasks
class NewsArticles(BaseModel):
    articles: list[str]

retrieve_news_task = Task(
    description="Fetch recent financial news articles about {company_name}",
    expected_output="A list of recent news articles.",
    output_json=NewsArticles,
    agent=news_retrieval_agent,
    tools=[news_tool],
)

sentiment_analysis_task = Task(
    description="Analyze the sentiment of the news articles for {company_name}",
    expected_output="A list of sentiment analysis results.",
    agent=sentiment_analysis_agent,
)

recommendation_task = Task(
    description="Provide an investment recommendation based on the sentiment analysis results", 
    expected_output="An investment recommendation for {company_name}",
    agent=recommendation_agent,
)

In [27]:
# Define the Crew and Assign Tasks
financial_analysis_crew = Crew(
    agents=[chatbot_agent, news_retrieval_agent, sentiment_analysis_agent, recommendation_agent],
    tasks=[retrieve_news_task, sentiment_analysis_task, recommendation_task],
    verbose=True,
)


2024-10-24 09:54:32,480 - 140704492328576 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [28]:
# Initiate the Process
company_name = "TSLA"
stock_ticker = chatbot_agent.get_stock_ticker(company_name)
if stock_ticker:
    print(f"Stock ticker for {company_name} is {stock_ticker}")
    result = financial_analysis_crew.kickoff(inputs={"company_name": company_name})
else:
    print("Invalid company name. Please try again.")

2024-10-24 09:54:33,777 - 140704492328576 - quote.py-quote:589 - ERROR: 404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/TESLA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=TESLA&crumb=i9feaweny0S


Error occurred while retrieving the stock ticker: 'symbol'
Invalid company name. Please try again.
